In [52]:
!pip install data_preprocessing
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
!pip install tensorflow


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0


In [54]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import *
import tensorflow as tf
import joblib, json
import pandas as pd
import numpy as np
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Model
from data_preprocessing import *
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, LSTM, Attention, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [55]:
df = pd.read_excel(open('/content/Consolidated.xlsx', 'rb'), sheet_name='Consolidated')

In [56]:
df_final_test = df.query('Foldername == "fold1"')

In [57]:
le = joblib.load('/content/labelEncoder.pkl')

In [58]:
num_classes = 4
embedding_dim = 128
num_heads = 2
max_seq_length = 250

In [59]:
def pre_process_data(X, y, max_seq, num_classes, token_model_save=True):
    tokenizer = Tokenizer()
    if token_model_save:
        tokenizer.fit_on_texts(X)
        with open('saved_models/tokenizer.pickle', 'wb') as f:
            pickle.dump(tokenizer, f)
    else:
        with open('saved_models/tokenizer.pickle', 'rb') as f:
            tokenizer = pickle.load(f)
    X_train_sequences = tokenizer.texts_to_sequences(X)
    max_seq_length = 250  # Choose the maximum sequence length
    X_train_padded = pad_sequences(X_train_sequences, maxlen=max_seq, padding='post')
    target_labels = tf.keras.utils.to_categorical(le.transform(y), num_classes)
    return X_train_padded, target_labels

In [60]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, RepeatVector, Input, Reshape, Flatten
from tensorflow.keras.models import *
import joblib
from xgboost import XGBClassifier

def preprocess_text(text):
    # 1. Lowercasing
    text = text.lower()

    # 2. Removing special characters & punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # 3. Tokenization
    tokens = nltk.word_tokenize(text)

    # 4. Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # 5. Lemmatization (Change words to root form)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # 6. Joining tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text






In [61]:
X, y = pre_process_data(df['Review'].apply(preprocess_text), df['Target'], max_seq_length, num_classes)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
def create_model(max_seq_length, embedding_dim, num_heads):
    input_layer = Input(shape=(max_seq_length,))
    embedding_layer = Embedding(input_dim=1500, output_dim=embedding_dim)(input_layer)
    attention_layer = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(embedding_layer, embedding_layer)
    pooling_layer = GlobalAveragePooling1D()(attention_layer)
    dropout_layer = Dropout(0.1)(pooling_layer)
    output_layer = Dense(4, activation='softmax')(dropout_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [64]:
model = create_model(max_seq_length, embedding_dim, num_heads)

In [65]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=tf.keras.metrics.F1Score(name='f1_score',average='macro'))

In [66]:
model.fit(X_train, y_train, epochs=10, batch_size=64, shuffle=True, validation_split=0.2)

Epoch 1/10
32/32 [==============================] - 7s 122ms/step - loss: 1.3770 - f1_score: 0.2595 - val_loss: 1.3405 - val_f1_score: 0.2033
Epoch 2/10
32/32 [==============================] - 3s 96ms/step - loss: 1.1814 - f1_score: 0.3218 - val_loss: 1.0122 - val_f1_score: 0.3451
Epoch 3/10
32/32 [==============================] - 4s 144ms/step - loss: 0.8607 - f1_score: 0.4999 - val_loss: 0.8148 - val_f1_score: 0.5136
Epoch 4/10
32/32 [==============================] - 5s 165ms/step - loss: 0.7530 - f1_score: 0.5408 - val_loss: 0.7628 - val_f1_score: 0.6139
Epoch 5/10
32/32 [==============================] - 4s 123ms/step - loss: 0.6889 - f1_score: 0.6184 - val_loss: 0.6831 - val_f1_score: 0.7257
Epoch 6/10
32/32 [==============================] - 3s 107ms/step - loss: 0.5309 - f1_score: 0.7992 - val_loss: 0.4869 - val_f1_score: 0.8277
Epoch 7/10
32/32 [==============================] - 2s 72ms/step - loss: 0.2824 - f1_score: 0.9017 - val_loss: 0.3404 - val_f1_score: 0.8546
Epoch 8/

In [67]:
predictions = model.predict(X_test)

20/20 [==============================] - 0s 4ms/step


In [68]:
f1_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1), average='macro')

0.9425526503456946

In [69]:
model.save('saved_models/transformer.keras')

## Test model with unseen data

In [70]:
t_X , t_y = pre_process_data(df_final_test['Review'].apply(preprocess_text), df_final_test['Target'], max_seq_length, num_classes, token_model_save=False)

In [71]:
l_model = tf.keras.models.load_model('saved_models/transformer.keras')
predictions = l_model.predict(t_X)

10/10 [==============================] - 0s 4ms/step


In [72]:
f1_score(np.argmax(t_y, axis=1), np.argmax(predictions, axis=1), average='macro')

0.962570819803974

In [73]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define hyperparameters to tune
optimizers = ['adam', 'rmsprop']
hidden_units = [32, 64, 128]

best_accuracy = 0
best_params = {}

for optimizer in optimizers:
    for units in hidden_units:
        # Create a Keras model with specified hyperparameters

        # Perform cross-validation
        kfold = KFold(n_splits=3, shuffle=True)
        accuracies = []
        for train_idx, val_idx in kfold.split(X_train):
            X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
            y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

            model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0)
            _, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
            accuracies.append(accuracy)

        mean_accuracy = np.mean(accuracies)
        print(f"Optimizer: {optimizer}, Hidden Units: {units}, Mean Accuracy: {mean_accuracy}")

        # Track the best hyperparameters
        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_params = {'optimizer': optimizer, 'hidden_units': units}

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)

Optimizer: adam, Hidden Units: 32, Mean Accuracy: 0.9925203919410706
Optimizer: adam, Hidden Units: 64, Mean Accuracy: 0.9956803917884827
Optimizer: adam, Hidden Units: 128, Mean Accuracy: 1.0
Optimizer: rmsprop, Hidden Units: 32, Mean Accuracy: 1.0
Optimizer: rmsprop, Hidden Units: 64, Mean Accuracy: 1.0
Optimizer: rmsprop, Hidden Units: 128, Mean Accuracy: 1.0
Best Parameters: {'optimizer': 'adam', 'hidden_units': 128}
Best Accuracy: 1.0


In [74]:
best_model = Sequential()
best_model.add(Dense(128, activation='relu', input_shape=(1500,)))
best_model.add(Dense(num_classes, activation='softmax'))
best_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=tf.keras.metrics.F1Score(name='f1_score',average='macro'))

In [75]:
model.fit(X_train, y_train, epochs=10, batch_size=64, shuffle=True, validation_split=0.2)

Epoch 1/10
32/32 [==============================] - 2s 64ms/step - loss: 7.1306e-07 - f1_score: 1.0000 - val_loss: 3.2666e-07 - val_f1_score: 1.0000
Epoch 2/10
32/32 [==============================] - 2s 59ms/step - loss: 5.7200e-07 - f1_score: 1.0000 - val_loss: 3.2387e-07 - val_f1_score: 1.0000
Epoch 3/10
32/32 [==============================] - 2s 48ms/step - loss: 6.9096e-07 - f1_score: 1.0000 - val_loss: 3.2200e-07 - val_f1_score: 1.0000
Epoch 4/10
32/32 [==============================] - 2s 69ms/step - loss: 4.6676e-07 - f1_score: 1.0000 - val_loss: 3.2037e-07 - val_f1_score: 1.0000
Epoch 5/10
32/32 [==============================] - 1s 41ms/step - loss: 5.2544e-07 - f1_score: 1.0000 - val_loss: 3.1944e-07 - val_f1_score: 1.0000
Epoch 6/10
32/32 [==============================] - 1s 41ms/step - loss: 5.1799e-07 - f1_score: 1.0000 - val_loss: 3.1572e-07 - val_f1_score: 1.0000
Epoch 7/10
32/32 [==============================] - 1s 26ms/step - loss: 6.1554e-07 - f1_score: 1.0000 - v

In [76]:
predictions = model.predict(X_test)

20/20 [==============================] - 0s 4ms/step


In [77]:
predictions = model.predict(X_test)

20/20 [==============================] - 0s 4ms/step


In [78]:
model.save('saved_models/transformer_updated.keras')

In [79]:
t_X , t_y = pre_process_data(df_final_test['Review'].apply(preprocess_text), df_final_test['Target'], max_seq_length, num_classes, token_model_save=False)

In [80]:
l_model = tf.keras.models.load_model('saved_models/transformer_updated.keras')
predictions = l_model.predict(t_X)

10/10 [==============================] - 0s 5ms/step


In [81]:
f1_score(np.argmax(t_y, axis=1), np.argmax(predictions, axis=1), average='macro')

0.9843746337747568